In [1]:
from osgeo import gdal
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import read_cover_fraction as rcf
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely import geometry

# Declare functions

## function to read the data

In [2]:
def get_dataSG(fname,latmax,latmin,lonmax,lonmin):

    gdal.UseExceptions()
    ds = gdal.Open(fname)
    data = ds.ReadAsArray()
    gt = ds.GetGeoTransform()
    #
    #
    #
    xres = gt[1]
    yres = gt[5]
    #
    xmin = gt[0]
    ymin = gt[3]
    #
    xmax = gt[0] + (xres * ds.RasterXSize)
    ymax = gt[3] + (yres * ds.RasterYSize)
#    else:
#        xmax = gt[0] + (xres * ds.RasterXSize) 
#        ymax = gt[3] + (yres * ds.RasterYSize) 


    X=np.arange(xmin+xres,xmax+xres,xres)
    Y=np.arange(ymin+yres,ymax+yres,yres)


    #Chunck the data to save RAM for plotting
    #Ydecreasing
    b=np.min(np.where(Y<latmin)[0])
    a=np.max(np.where(Y>latmax)[0])

    c=np.max(np.where(X<lonmin)[0])
    d=np.min(np.where(X>lonmax)[0])

#    print(np.max(data))
#    print(np.min(data))
    #ECOCLIMAP data have no projections so no need to reproject the data
    #to be verified for other datasets
    LAT=Y[a:b]
    LON=X[c:d]
    D=data[a:b,c:d]

    return (LAT,LON,D)

## Read station coordinates function

In [3]:
def readstationcoordinate(fnstation):
    data=open(fnstation,'r')
    lat=[]
    lon=[]
    for d in data:
        lat.append(d.split(' ')[0].strip(' '))
        lon.append(d.split(' ')[1].strip(' '))
    LAT=np.array(lat).astype(float)
    LON=np.array(lon).astype(float)
    return(LAT,LON)

## Function to set correspondance between station and line in the csv files

In [4]:
def set_covernumber_station(LAT,LON,LATSG,LONSG,covernum,DSG):
    Linenum=[]
    for x,_ in enumerate(LAT):
        idx=np.argmin(np.abs(LAT[x]-LATSG))
        idy=np.argmin(np.abs(LON[x]-LONSG))
        Linenum.append(np.where(covernum==int(DSG[idx,idy]))[0][0])
    return (Linenum)

## Write cover station text file

In [5]:
def write_txt_file_cover_station(fntxt,LAT,LON,Linenum,covernum,covername,cityfrac,vegfrac,inwaterfrac,seafrac):
    data=open(fntxt,'w')
    data.write('LAT, LON, Covernum, Covername, City fraction, Vegetation fraction, Inland water fraction, sea fraction\n')
    for x,_ in enumerate(LAT):
        data.write(str(LAT[x])+', '+str(LON[x])+', '+str(covernum[Linenum[x]])+', '+covername[Linenum[x]]+', '+
                   str(cityfrac[Linenum[x]])+', '+str(vegfrac[Linenum[x]])+', '+
                   str(inwaterfrac[Linenum[x]])+', '+str(seafrac[Linenum[x]])+'\n')
    data.close()
    return

## Function to extract the Surfex data

In [6]:
def surfex_fractions (fnforc,fn,Variablelist):
    FORC=xr.open_dataset(fnforc)
    OUT=xr.open_dataset(fn)
    x_shape=int(OUT['IMAX'].values.mean())
    y_shape=int(OUT['JMAX'].values.mean())
    Pn=OUT.PATCH.Number_of_Patches
    df=pd.DataFrame()
    df['Geometry']=[geometry.Point(x, y) for x, y in zip(FORC['LAT'], FORC['LON'])]
    gdf=gpd.GeoDataFrame(df,geometry='Geometry', crs= 'EPSG:4326')
    LEG=[]
    LEG.append('Geometry')
    for Variable in Variablelist:
        if len(OUT[Variable].shape)==3:
            for P in Pn: 
                LEG.append(OUT[Variable].comment)
                vname=Variable+'Pa'+str(int(np.mean(P.values))+1)
                gdf[vname]=OUT[Variable][int(np.mean(P.values))].to_dataframe(name=vname).values
        else:
            gdf[Variable]=OUT[Variable].to_dataframe(name=Variable).values
            LEG.append(OUT[Variable].comment)
    return (x_shape,y_shape,gdf,LEG)

## Get lat/lon for plotting

In [7]:
def getlatlon1d(gdf,x_shape,y_shape):
    gdf['LONGITUDE']=[d.y for d in gdf.geometry]
    gdf['LATITUDE']=[d.x for d in gdf.geometry]
    LON=gdf.LONGITUDE.values.reshape(x_shape,y_shape)
    LAT=gdf.LATITUDE.values.reshape(x_shape,y_shape)
    LAT1d=LAT[0,:]
    LON1d=LON[:,0]
    return (LON1d,LAT1d)

## Plot surfex data

In [8]:
def surfex_plot_ecosg_eco2(fnforc,fneco2,fnSG,Variableslist='Z0',colormap='jet',VMIN=0,VMAX=1,
                          latc=53.2,lonc=-8.2,extent=4.6,stationno='00'):
    if type(Variableslist)!=list:
        Variableslist=[Variableslist]
    (x_shape,y_shape,gdfeco2,LEGeco2)=surfex_fractions(fnforc,fneco2,Variableslist)
    (x_shape,y_shape,gdfSG,LEGSG)=surfex_fractions(fnforc,fnSG,Variableslist)
    (LON1d2,LAT1d2)=getlatlon1d(gdfeco2,x_shape,y_shape)
    (LON1dsg,LAT1dsg)=getlatlon1d(gdfSG,x_shape,y_shape)
    if type(latc)!=list:
        latc=[latc]
        lonc=[lonc]
        stationno=[stationno]
    for j,_ in enumerate(latc):
        latmin=latc[j]-extent
        latmax=latc[j]+extent
        lonmin=lonc[j]-extent
        lonmax=lonc[j]+extent
        for i,vname in enumerate(gdfeco2.columns):
            if 'Geometry' not in vname:
                if 'LONGITUDE' not in vname:
                    if 'LATITUDE' not in vname:
                        fig,axes=plt.subplots(1,2,figsize=(15*2,15))
                        c=axes[0].pcolormesh(LON1dsg,LAT1dsg,gdfSG[vname].values.reshape(x_shape,y_shape),cmap=colormap,vmin=VMIN,vmax=VMAX)
                        axes[0].set_xlabel('Longitude',fontsize=20)
                        axes[0].set_ylabel('Latitude',fontsize=20)
                        axes[0].xaxis.set_tick_params(labelsize=20)
                        axes[0].yaxis.set_tick_params(labelsize=20)
                        axes[0].set_xlim(lonmin,lonmax)
                        axes[0].set_ylim(latmin,latmax)
                        axes[0].set_title(vname+' ECOCLIMAPSG',fontsize=20)
                        c=axes[1].pcolormesh(LON1d2,LAT1d2,gdfeco2[vname].values.reshape(x_shape,y_shape),cmap=colormap,vmin=VMIN,vmax=VMAX)
                        axes[1].set_xlabel('Longitude',fontsize=20)
                        axes[1].set_ylabel('Latitude',fontsize=20)
                        axes[1].xaxis.set_tick_params(labelsize=20)
                        axes[1].yaxis.set_tick_params(labelsize=20)
                        axes[1].set_xlim(lonmin,lonmax)
                        axes[1].set_ylim(latmin,latmax)
                        axes[1].set_title(vname+' ECOCLIMAP2',fontsize=20)
                        cbar_ax = fig.add_axes([0.91, 0.10, 0.03, 0.8])
                        cb=fig.colorbar(c, cax=cbar_ax) 
                        cb.set_label(LEGeco2[i],fontsize=20)
                        cb.ax.tick_params(labelsize=20)
                        fig.savefig(vname+'latc'+str(latc[j])+'lonc'+str(lonc[j])+'sta'+str(stationno[j])+'surfex.png')
                        plt.close(fig)
    return

In [9]:
def plot_ecosg_eco2_fraction(fractionsg,LONsg,LATsg,fractionII,LONII,LATII,colormap='jet',VMIN=0,VMAX=1,
                          latc=53.2,lonc=-8.2,extent=4.6,stationno='00',LEG='Fraction',vname='Fraction'):
#function to plot ecoclimap2 and ecococlimap SG fractions in its native resolution
#fractionsg fraction data ecoclimap sg
#LONsg.LATsg longitude latitude ecoclimap SG
#fractionII fraction data ecoclimap 2
#LONII,LATII latitude longitude ecoclimap2 
#VMIN colorbar min
#VMAX colorbar max
#vname variable name for filename save
#LEG colorbar legend char
#latc,lonc longitude latitude center of the plot
#extent extent of the plot
# stationno station num or name for filesave
    if type(latc)!=list:
        latc=[latc]
        lonc=[lonc]
        stationno=[stationno]
    for j,_ in enumerate(latc):
        latmin=latc[j]-extent
        latmax=latc[j]+extent
        lonmin=lonc[j]-extent
        lonmax=lonc[j]+extent
        fig, axes = plt.subplots(1, 2,figsize=(25, 25))
        c=axes[0].pcolormesh(LONsg,LATsg,fractionsg,cmap=colormap, vmin=VMIN,vmax=VMAX)
        axes[0].set_xlabel('Longitude',fontsize=20)
        axes[0].set_ylabel('Latitude',fontsize=20)
        axes[0].xaxis.set_tick_params(labelsize=20)
        axes[0].yaxis.set_tick_params(labelsize=20)
        axes[0].set_xlim(lonmin,lonmax)
        axes[0].set_ylim(latmin,latmax)
        axes[0].set_title(vname+' ECOCLIMAPSG',fontsize=20)
        c=axes[1].pcolormesh(LONII,LATII,fractionII,cmap=colormap,vmin=VMIN,vmax=VMAX)
        axes[1].set_xlabel('Longitude',fontsize=20)
        axes[1].set_ylabel('Latitude',fontsize=20)
        axes[1].xaxis.set_tick_params(labelsize=20)
        axes[1].yaxis.set_tick_params(labelsize=20)
        axes[1].set_xlim(lonmin,lonmax)
        axes[1].set_ylim(latmin,latmax)
        axes[1].set_title(vname+' ECOCLIMAP2',fontsize=20)
        cbar_ax = fig.add_axes([0.91, 0.10, 0.03, 0.8])
        cb=fig.colorbar(c, cax=cbar_ax) 
        cb.set_label(LEG,fontsize=20)
        cb.ax.tick_params(labelsize=20)
        fig.savefig(vname+'latc'+str(latc[j])+'lonc'+str(lonc[j])+'sta'+str(stationno[j])+'native.png')
        plt.close(fig)
    return

# MAIN 

## Airport station coordinate

In [10]:
#Cork airport
corklat=51.842
corklon=-8.485
#Dublin airport
dubalat=53.423 
dubalon=-6.238
#Knock airport
Knlat=53.906
Knlon=-8.817
#Shannon airport
Shlat=52.687
Shlon=-8.917
# Sligo airport
Sllat=54.274
Sllon=-8.592
LAT=[corklat,dubalat,Knlat,Shlat,Sllat]
LON=[corklon,dubalon,Knlon,Shlon,Sllat]
AIRPORT=['Cork','Dublin','Knock','Shannon','Sligo']

## Open ECOCLIMAP-SG data

In [11]:
latmax=55.5
latmin=50.9
lonmax=-5.4
lonmin=-11

fnameSG='/mnt/g/ECOCLIMAP/ECOCLIMAPSG/test_ecosg.tif'
(LATSG,LONSG,DSG)=get_dataSG(fnameSG,latmax,latmin,lonmax,lonmin)
covernamef='/mnt/g/ECOCLIMAP/ECOCLIMAPSG/ECOCLIMAP_SG_cover_data.csv'
#get the cover deatils from the csvfile
(fieldnames,covernum,covername,cityfrac,vegfrac,inwaterfrac,seafrac)=rcf.cover_fraction(covernamef)
#create fraction array 
TownfracSG=rcf.convert_to_cover_fraction(DSG,cityfrac)
VegfracSG=rcf.convert_to_cover_fraction(DSG,vegfrac)
InwfracSG=rcf.convert_to_cover_fraction(DSG,inwaterfrac)
SeafracSG=rcf.convert_to_cover_fraction(DSG,seafrac)

## Open ECOCLIMAP II data

In [12]:
latmax=55.5
latmin=50.9
lonmax=-5.4
lonmin=-11

fnameII='/mnt/g/ECOCLIMAP/open_ECOCLIMAP2.3/ECOCLIMAP_II_EUROP_V2.3.dir'
(LATII,LONII,DII)=get_dataSG(fnameII,latmax,latmin,lonmax,lonmin)
filenamecoverII='/mnt/g/ECOCLIMAP/ECOCLIMAP_II_cover_data.csv'
(fieldnamesII,covernumII,covernameII,cityfracII,vegfracII,inwaterfracII,seafracII)=rcf.cover_fraction(filenamecoverII)
#create fraction array 
TownfracII=rcf.convert_to_cover_fraction(DII,cityfracII)
VegfracII=rcf.convert_to_cover_fraction(DII,vegfracII)
InwfracII=rcf.convert_to_cover_fraction(DII,inwaterfracII)
SeafracII=rcf.convert_to_cover_fraction(DII,seafracII)

## Open and plot offline-surfex data

In [13]:
Variableslist=['FRAC_SEA','FRAC_NATURE','FRAC_WATER','FRAC_TOWN','PATCH','Z0','Z0_TEB','Z0_ISBA']
fnforc='/mnt/g/offline_surfex_MERA/FORCING.nc'
fnSG='/home/geoffreybess/SURFEX/open_SURFEX_V8_1/MY_RUN/Eire/SURFOUT.20180618_07h00.nc'
fneco2='/home/geoffreybess/SURFEX/open_SURFEX_V8_1/MY_RUN/Eireeco2/SURFOUT.20180618_07h00.nc'

surfex_plot_ecosg_eco2(fnforc,fneco2,fnSG,Variableslist,latc=LAT,lonc=LON,extent=0.05,stationno=AIRPORT)

## Plot fraction cover

In [14]:
#Airports or station
plot_ecosg_eco2_fraction(TownfracSG,LONSG,LATSG,TownfracII,LONII,LATII,latc=LAT,lonc=LON,extent=0.05,
                         stationno=AIRPORT,LEG='Town Fraction',vname='Town_Fraction')
plot_ecosg_eco2_fraction(VegfracSG,LONSG,LATSG,VegfracII,LONII,LATII,latc=LAT,lonc=LON,extent=0.05,
                         stationno=AIRPORT,LEG='Nature Fraction',vname='Nature_Fraction')
plot_ecosg_eco2_fraction(InwfracSG,LONSG,LATSG,InwfracII,LONII,LATII,latc=LAT,lonc=LON,extent=0.05,
                         stationno=AIRPORT,LEG='Inland water Fraction',vname='Inland_water_Fraction')
plot_ecosg_eco2_fraction(SeafracSG,LONSG,LATSG,SeafracII,LONII,LATII,latc=LAT,lonc=LON,extent=0.05,
                         stationno=AIRPORT,LEG='Sea Fraction',vname='Sea_Fraction')


#Nationwide
plot_ecosg_eco2_fraction(TownfracSG,LONSG,LATSG,TownfracII,LONII,LATII,LEG='Town Fraction',vname='Town_Fraction')
plot_ecosg_eco2_fraction(VegfracSG,LONSG,LATSG,VegfracII,LONII,LATII,LEG='Nature Fraction',vname='Nature_Fraction')
plot_ecosg_eco2_fraction(InwfracSG,LONSG,LATSG,InwfracII,LONII,LATII,LEG='Inland water Fraction',vname='Inland_water_Fraction')
plot_ecosg_eco2_fraction(SeafracSG,LONSG,LATSG,SeafracII,LONII,LATII,LEG='Sea Fraction',vname='Sea_Fraction')
